In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm
import json
from collections import Counter
import random
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Import distance limits


In [ ]:
#Import distance limits
distance_limits = pd.read_csv ('drive/MyDrive/Amenities/data/distance_to_amenities.csv', index_col = False)
distance_limits.head(2)

,sub_amenity,distance (miles)
0,Full-Service Restaurants,0.23
1,Snack and Nonalcoholic Beverage Bars,0.26


# Import trips

In [ ]:
##Import trip info
trips_sa = pd.read_csv ('drive/MyDrive/Amenities/output/BUA_trips_bga_sa_amenitiesadded2.csv', index_col = False)
trips_th = pd.read_csv ('drive/MyDrive/Amenities/output/BUA_trips_bga_th_amenitiesadded2.csv', index_col = False)
trips_sa.head(2)

,activity_id,person_id,household_id,mode,duration_minutes,distance_miles,origin_bgrp,destination_bgrp,naics_category,placekey,naics_code,location_name,top_category,sub_category,vehicles,sub_amenity,naics_4,travel_purpose,top_amenity
0,1.415185e+19,3.820938e+18,4.456505e+18,CARPOOL,111.0,101.180729,2.300502e+11,2.502507e+11,71,239-222@62j-sj3-sqz,713940.0,Balance At Urbanity Dance,Other Amusement and Recreation Industries,Fitness and Recreational Sports Centers,1,Gym,7139,Recreation,Leisure & Wellness
1,5.174423e+18,1.503045e+19,1.449734e+19,PUBLIC_TRANSIT,59.0,7.325792,2.502505e+11,2.502598e+11,71,zzw-222@62j-sck-j7q,712190.0,Franklin Park,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,0,Zoos and Nature Parks,7121,Recreation,Entertainment


In [ ]:
trips_th = trips_th.replace('Professional services', 'Professional Services')
trips_sa = trips_sa.replace('Professional services', 'Professional Services')
trips_th = trips_th.replace('Limited-Service Restaurants ', 'Limited-Service Restaurants')
trips_sa = trips_sa.replace('Limited-Service Restaurants ', 'Limited-Service Restaurants')
trips_th = trips_th.replace('Drinking Places ', 'Drinking Places')
trips_sa = trips_sa.replace('Drinking Places ', 'Drinking Places')
trips_th = trips_th.replace('Snack and Nonalcoholic Beverage Bars ', 'Snack and Nonalcoholic Beverage Bars')
trips_sa = trips_sa.replace('Snack and Nonalcoholic Beverage Bars ', 'Snack and Nonalcoholic Beverage Bars')
trips_th = trips_th.replace('Specialty Food Retailers ', 'Specialty Food Retailers')
trips_sa = trips_sa.replace('Specialty Food Retailers ', 'Specialty Food Retailers')

In [ ]:
trips_sa['mode'].unique()

array(['CARPOOL', 'PUBLIC_TRANSIT', 'ON_DEMAND_AUTO', 'OTHER_TRAVEL_MODE',
       'WALKING', 'BIKING', 'PRIVATE_AUTO'], dtype=object)

In [ ]:
cond1 = {
    'BIKING': 'Biking',
    'CARPOOL':'Car',
    'ON_DEMAND_AUTO':'Car',
    'PRIVATE_AUTO':'Car',
    'PUBLIC_TRANSIT':'Public_transport',
    'WALKING':'Walking'
}

trips_sa['mode'] = trips_sa['mode'].map(cond1)
trips_th['mode'] = trips_th['mode'].map(cond1)
trips_sa=trips_sa[trips_sa['mode'].notnull()]
trips_th=trips_th[trips_th['mode'].notnull()]

trips_sa.sort_index(axis = 1)
trips_th.sort_index(axis = 1)

,activity_id,destination_bgrp,distance_miles,duration_minutes,household_id,location_name,mode,naics_4,naics_category,naics_code,origin_bgrp,person_id,placekey,sub_amenity,sub_category,top_amenity,top_category,travel_purpose,vehicles
0,5.171354e+18,2.502508e+11,1.493868,6.0,1.163440e+19,M & M Ribs,Car,7223,72,722320.0,2.502506e+11,7.734123e+18,zzw-222@62j-sjp-qj9,Special Food Services,Caterers,F&B Eatery,Special Food Services,Eat,3
1,1.678649e+19,2.502508e+11,5.193408,17.0,1.581380e+19,Caffè Nero,Car,7225,72,722515.0,2.502504e+11,1.748523e+18,22b-226@62j-sgf-j9z,Snack and Nonalcoholic Beverage Bars,Snack and Nonalcoholic Beverage Bars,F&B Eatery,Restaurants and Other Eating Places,Eat,2
2,1.026687e+19,2.501735e+11,3.948695,21.0,4.376605e+18,Mainely Burgers,Biking,7225,72,722511.0,2.501735e+11,7.278027e+18,22y-222@62j-sj2-t35,Full-Service Restaurants,Full-Service Restaurants,F&B Eatery,Restaurants and Other Eating Places,Eat,1
3,5.553640e+18,2.502512e+11,0.127681,2.0,1.381520e+19,Tavern Club,Walking,7225,72,722511.0,2.502512e+11,7.098890e+18,24g-222@62j-sg9-ks5,Full-Service Restaurants,Full-Service Restaurants,F&B Eatery,Restaurants and Other Eating Places,Eat,0
4,1.672726e+19,2.502501e+11,1.356521,4.0,1.818483e+19,Rhett's,Car,7225,72,722511.0,2.502500e+11,1.769660e+19,222-225@62j-sgf-fmk,Full-Service Restaurants,Full-Service Restaurants,F&B Eatery,Restaurants and Other Eating Places,Eat,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210388,1.368682e+19,2.502500e+11,0.061125,1.0,2.987716e+18,Chonga Fanwei Circular Kung Fu Academy,Walking,6116,61,611620.0,2.502501e+11,1.832456e+19,222-223@62j-sgf-syv,Sports and Recreation Instruction,Sports and Recreation Instruction,Education,Other Schools and Instruction,School,0
210389,1.010424e+19,2.502501e+11,1.177918,17.0,1.048480e+19,Kids Are People Elementary School,Public_transport,6111,61,611110.0,2.502507e+11,1.592686e+19,23h-222@62j-sgg-d35,Elementary School,Elementary and Secondary Schools,Education,Elementary and Secondary Schools,School,2
210391,1.277694e+19,2.502505e+11,30.314164,39.0,1.593482e+19,Einstein Tutoring,Car,6116,61,611691.0,2.500925e+11,4.192011e+18,zzw-222@62j-sj6-xwk,Other Schools and Instruction,Exam Preparation and Tutoring,Education,Other Schools and Instruction,School,1
210393,1.493686e+19,2.502501e+11,0.184493,3.0,3.824557e+18,Simmons College Residence Campus,Walking,6113,61,611310.0,2.502501e+11,9.535223e+18,22b-222@62j-sgf-mp9,Universities,"Colleges, Universities, and Professional Schools",Education,"Colleges, Universities, and Professional Schools",School,0


# Filter trips

In [ ]:
#Merge trips to form a week
trips_week = pd.concat([trips_th, trips_th, trips_th, trips_th, trips_th, trips_sa, trips_sa]).reset_index().drop('index', axis = 1)

#Filter negative values and outliers (z > 3)
trips_aux = trips_week[(trips_week['distance_miles']>0)]
trips_aux = trips_week[(trips_week['duration_minutes']>0)]

#Get an array with the travel purpose names
travel_purpose_names=trips_week['top_amenity'].unique()

#Filter the outliers of the geometry file
trips_filt=pd.DataFrame()

for i in range(len(travel_purpose_names)):
  #Extract the travel purpose name from the array
  travel_purpose=travel_purpose_names[i]
  #Extract the trips using with that travel purpose
  trips_aux_2=trips_aux[trips_aux['top_amenity']==travel_purpose]

  #Get an array with the transport mode names
  transport_mode_names=trips_aux_2['mode'].unique()

  for i in range(len(transport_mode_names)):
    #Extract the transport mode name from the array
    transport_mode=transport_mode_names[i]
    #Extract the trips using that mode of transport
    trips_trans=trips_aux_2[trips_aux_2['mode']==transport_mode].reset_index()
    #Calculate the  scores
    trips_trans['z_scores_miles'] =stats.zscore(trips_trans['distance_miles'])
    trips_trans['z_scores_time'] =stats.zscore(trips_trans['duration_minutes'])

    #Take out the outliers
    trips_temp = trips_trans[np.abs(trips_trans['z_scores_miles']) < 3].reset_index().drop('index', axis = 1)
    trips_temp = trips_trans[np.abs(trips_trans['z_scores_time']) < 3].reset_index().drop('index', axis = 1)
    #Make a dataframe with the filtered trips
    trips_filt=pd.concat([trips_filt, trips_temp])

trips_filt = trips_filt.drop('level_0', axis=1)
trips_filt.head(2)

,activity_id,person_id,household_id,mode,duration_minutes,distance_miles,origin_bgrp,destination_bgrp,naics_category,placekey,...,location_name,top_category,sub_category,vehicles,sub_amenity,naics_4,travel_purpose,top_amenity,z_scores_miles,z_scores_time
0,5.171354e+18,7.734123e+18,1.163440e+19,Car,6.0,1.493868,2.502506e+11,2.502508e+11,72,zzw-222@62j-sjp-qj9,...,M & M Ribs,Special Food Services,Caterers,3,Special Food Services,7223,Eat,F&B Eatery,-0.264011,-0.264840
1,1.678649e+19,1.748523e+18,1.581380e+19,Car,17.0,5.193408,2.502504e+11,2.502508e+11,72,22b-226@62j-sgf-j9z,...,Caffè Nero,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,2,Snack and Nonalcoholic Beverage Bars,7225,Eat,F&B Eatery,0.123101,0.561857


# Travel choice with defined distance limits


In [ ]:
travel_choice_dLimits=pd.DataFrame()
trips_with_dlim=pd.DataFrame()

for i in range(len(distance_limits['sub_amenity'].unique())):

  sub_amenity=distance_limits['sub_amenity'][i]
  d_lim=distance_limits['distance (miles)'][i]

  #print (sub_amenity)

  list_trips_sub=trips_filt[trips_filt['sub_amenity']==sub_amenity].reset_index(drop=True)
  list_trips_sub=list_trips_sub[list_trips_sub['distance_miles']<d_lim].reset_index(drop=True)
  trips_with_dlim=pd.concat([trips_with_dlim, list_trips_sub])

  list_trips_sub=list_trips_sub.groupby(['sub_amenity', 'mode']).agg({'activity_id': 'count'}).reset_index().rename(columns = {'activity_id': 'n_trips'})
  count_purpose = list_trips_sub.groupby(['sub_amenity'])['n_trips'].sum()
  list_trips_sub['% trips']=list_trips_sub['n_trips']/count_purpose[0]*100

  list_trips_transpose = pd.pivot_table(data=list_trips_sub, index=['sub_amenity'], columns=['mode'], values='% trips')
  travel_choice_dLimits=pd.concat([travel_choice_dLimits, list_trips_transpose])

travel_choice_dLimits.head(5)

mode,Biking,Car,Public_transport,Walking
sub_amenity,,,,
Full-Service Restaurants,0.718839,1.141686,0.016263,98.123211
Snack and Nonalcoholic Beverage Bars,1.267322,2.297761,0.011844,96.423072
Limited-Service Restaurants,1.089310,2.670779,0.032811,96.207100
Grocery or Supermarket,0.669286,5.330035,0.383142,93.617537
Home Goods Store,0.696594,5.656605,0.270898,93.375903


In [ ]:
travel_choice_dLimits.reset_index(inplace=True)
travel_choice_dLimits = travel_choice_dLimits.rename(columns = {'index':'sub_amenity'})
travel_choice_dLimits.head(5)

mode,sub_amenity,Biking,Car,Public_transport,Walking
0,Full-Service Restaurants,0.718839,1.141686,0.016263,98.123211
1,Snack and Nonalcoholic Beverage Bars,1.267322,2.297761,0.011844,96.423072
2,Limited-Service Restaurants,1.089310,2.670779,0.032811,96.207100
3,Grocery or Supermarket,0.669286,5.330035,0.383142,93.617537
4,Home Goods Store,0.696594,5.656605,0.270898,93.375903


In [ ]:
travel_choice_dLimits.to_csv('drive/MyDrive/Amenities/output/validar_distanceLimits.csv', encoding='utf-8', index=False)

In [ ]:
trips_with_dlim.head(2)

,activity_id,person_id,household_id,mode,duration_minutes,distance_miles,origin_bgrp,destination_bgrp,naics_category,placekey,...,location_name,top_category,sub_category,vehicles,sub_amenity,naics_4,travel_purpose,top_amenity,z_scores_miles,z_scores_time
0,4.317615e+18,1.805957e+19,1.316331e+19,Car,1.0,0.223179,2.502140e+11,2.502140e+11,72,24m-222@62j-sgf-y35,...,BAB Korean Bistro,Restaurants and Other Eating Places,Full-Service Restaurants,0,Full-Service Restaurants,7225,Eat,F&B Eatery,-0.396973,-0.640612
1,7.646784e+18,8.442307e+18,5.166339e+18,Car,1.0,0.215750,2.502501e+11,2.502501e+11,72,zzy-222@62j-sj3-nwk,...,Eddie V's Prime Seafood,Restaurants and Other Eating Places,Full-Service Restaurants,1,Full-Service Restaurants,7225,Eat,F&B Eatery,-0.397751,-0.640612


In [ ]:
count_trips_by_mode = trips_with_dlim.groupby(['mode']).agg({'activity_id': 'count'}).reset_index().rename(columns = {'activity_id': 'n_trips'})
count_trips_by_mode

,mode,n_trips
0,Biking,2848
1,Car,15564
2,Public_transport,1113
3,Walking,313341


In [ ]:
#I am using this as the optimal case
count_trips_by_mode['% trips']=count_trips_by_mode['n_trips']/count_trips_by_mode['n_trips'].sum()*100
count_trips_by_mode

,mode,n_trips,% trips
0,Biking,2848,0.855600
1,Car,15564,4.675755
2,Public_transport,1113,0.334369
3,Walking,313341,94.134276


# Current travel choice


In [ ]:
#Count how many people use each transport mode depending on the travel purpose
count_transport_mode_sa = trips_sa.groupby(['sub_amenity', 'mode']).agg({'activity_id': 'count'}).reset_index().rename(columns = {'activity_id': 'n_trips'})
count_transport_mode_th = trips_th.groupby(['sub_amenity', 'mode']).agg({'activity_id': 'count'}).reset_index().rename(columns = {'activity_id': 'n_trips'})

#Merge them to form a week
count_transport_mode_sa['n_trips']=2*count_transport_mode_sa['n_trips']
count_transport_mode_th['n_trips']=5*count_transport_mode_th['n_trips']

count_transport_mode_week = pd.concat([count_transport_mode_th, count_transport_mode_sa]).reset_index().drop('index', axis = 1)
count_transport_mode_week = count_transport_mode_week.groupby(['sub_amenity', 'mode'])['n_trips'].sum().reset_index()

#Merge them to form a month
count_transport_mode_month = count_transport_mode_week.copy()
count_transport_mode_month['n_trips'] = round(count_transport_mode_week['n_trips']*52/12,0) # weeks in a year divided by the number of months
count_transport_mode_month.head(5)

,sub_amenity,mode,n_trips
0,Administrative and Support Services,Car,147.0
1,Administrative and Support Services,Public_transport,43.0
2,Administrative and Support Services,Walking,95.0
3,Amusement Park,Biking,30.0
4,Amusement Park,Car,1872.0


In [ ]:
#Count number of trips to each travel purpose
count_purpose = count_transport_mode_month.groupby(['sub_amenity'])['n_trips'].sum().reset_index().rename(columns = {'n_trips': 'n_visits'})
count_purpose.head(2)

,sub_amenity,n_visits
0,Administrative and Support Services,285.0
1,Amusement Park,4602.0


In [ ]:
total_trips=count_purpose['n_visits'].sum()
total_trips

7385232.0

In [ ]:
count_trips_by_mode= count_transport_mode_month.groupby(['mode'])['n_trips'].sum().reset_index()
count_trips_by_mode['% trips']=count_trips_by_mode['n_trips']/total_trips*100
count_trips_by_mode

,mode,n_trips,% trips
0,Biking,119570.0,1.619042
1,Car,2975600.0,40.291219
2,Public_transport,1456962.0,19.728046
3,Walking,2833100.0,38.361693


In [ ]:
#Merge tranport use and travel purpose to get percentage of transport modes depending on the travel purpose
analysis = count_transport_mode_month.merge(count_purpose, on='sub_amenity', how = 'left')
analysis['% trips']=analysis['n_trips']/analysis['n_visits']*100
analysis.head(2)

,sub_amenity,mode,n_trips,n_visits,% trips
0,Administrative and Support Services,Car,147.0,285.0,51.578947
1,Administrative and Support Services,Public_transport,43.0,285.0,15.087719


In [ ]:
#trips_filt = pd.read_csv ('drive/MyDriveAmenities/output/trips_time_distance_filt.csv', index_col = False)

In [ ]:
#Calculate mean distance for each the travel purpose
mean_distance = trips_filt.groupby(['sub_amenity']).agg({'distance_miles': 'mean'}).reset_index().rename(columns = {'distance_miles': 'mean_distance'})
mean_distance.head(5)

,sub_amenity,mean_distance
0,Administrative and Support Services,1.973972
1,Amusement Park,2.063398
2,Art Gallery,1.794481
3,Book Store,1.493061
4,Bowling,2.649085


In [ ]:
#Calculate mean distance for each transport mode depending on the travel purpose
mean_distance_by_mode = trips_filt.groupby(['sub_amenity', 'mode']).agg({'distance_miles': 'mean'}).reset_index().rename(columns = {'distance_miles': 'mean_distance'})
mean_distance_by_mode.head(5)

,sub_amenity,mode,mean_distance
0,Administrative and Support Services,Car,2.266045
1,Administrative and Support Services,Public_transport,4.480614
2,Administrative and Support Services,Walking,0.383203
3,Amusement Park,Biking,1.440030
4,Amusement Park,Car,2.949236


In [ ]:
#Calculate mean time travel for each transport mode depending on the travel purpose
mean_traveltime_by_mode = trips_filt.groupby(['sub_amenity', 'mode']).agg({'duration_minutes': 'mean'}).reset_index().rename(columns = {'duration_minutes': 'mean_traveltime'})
mean_traveltime_by_mode.head(5)

,sub_amenity,mode,mean_traveltime
0,Administrative and Support Services,Car,6.823529
1,Administrative and Support Services,Public_transport,33.000000
2,Administrative and Support Services,Walking,6.681818
3,Amusement Park,Biking,7.571429
4,Amusement Park,Car,8.166247


In [ ]:
#Merge tranport use and travel purpose to get percentage of transport modes depending on the travel purpose
analysis = analysis.merge(mean_distance_by_mode, on=['sub_amenity', 'mode'], how = 'left')
analysis = analysis.merge(mean_traveltime_by_mode, on=['sub_amenity', 'mode'], how = 'left')
analysis.head(2)

,sub_amenity,mode,n_trips,n_visits,% trips,mean_distance,mean_traveltime
0,Administrative and Support Services,Car,147.0,285.0,51.578947,2.266045,6.823529
1,Administrative and Support Services,Public_transport,43.0,285.0,15.087719,4.480614,33.000000


In [ ]:
pivot_analysis = pd.pivot_table(data=analysis, index=['sub_amenity'], columns=['mode'], values='% trips')
pivot_analysis

mode,Biking,Car,Public_transport,Walking
sub_amenity,,,,
Administrative and Support Services,NaN,51.578947,15.087719,33.333333
Amusement Park,0.651890,40.677966,25.619296,33.050847
Art Gallery,0.994475,30.706899,17.163904,51.134722
Book Store,1.410163,28.401504,12.654996,57.533337
Bowling,0.650842,42.457887,18.912711,37.978560
...,...,...,...,...
Veterinary Services,4.524887,60.180995,5.882353,29.411765
Warehouse,1.635033,38.562390,21.537465,38.265112
Waste Management and Remediation Services,0.773782,38.279472,14.610833,46.335913


In [ ]:
pivot_analysis.reset_index(inplace=True)
pivot_analysis = pivot_analysis.rename(columns = {'index':'sub_amenity'})
pivot_analysis

mode,sub_amenity,Biking,Car,Public_transport,Walking
0,Administrative and Support Services,NaN,51.578947,15.087719,33.333333
1,Amusement Park,0.651890,40.677966,25.619296,33.050847
2,Art Gallery,0.994475,30.706899,17.163904,51.134722
3,Book Store,1.410163,28.401504,12.654996,57.533337
4,Bowling,0.650842,42.457887,18.912711,37.978560
...,...,...,...,...,...
78,Veterinary Services,4.524887,60.180995,5.882353,29.411765
79,Warehouse,1.635033,38.562390,21.537465,38.265112
80,Waste Management and Remediation Services,0.773782,38.279472,14.610833,46.335913
81,Wholesale Trade,0.923164,52.435734,20.920324,25.720778


In [ ]:
pivot_analysis.to_csv('drive/MyDrive/Amenities/output/travel_choice_per_amenity.csv', encoding='utf-8', index=False)